### Stable Diffusion Pipeline (GPU)

##### https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/img2img

In [1]:
# system
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../..')))

from config.path import get_stable_input_path, get_stable_output_path
import torch
import os
from PIL import Image
import datetime
from diffusers import StableDiffusionImg2ImgPipeline
import random


device = "cuda" # this works gpu only
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

PATH_TO_STABLE_INPUT = get_stable_input_path()
PATH_TO_STABLE_OUTPUT = get_stable_output_path()

In [2]:
# define cloth type
clothType = 'mixed_'

prompts = [
        f'generated two clothes'
        #f'Transform this {clothType}, add new color and material, add crumpled and wrinkled up, add white seamless background'
        
        # f'{clothType} in random layout',
        # f'single {clothType} with pockets laying on white plain background'
        # f'single {clothType} with zipper on white plain background',
        # f'single {clothType} with knobs on white plain background'
        # f'single {clothType} with print on white plain background'
        # f'make this {clothType} look from the backside',
]

negative_prompt = ['hands, face, hair, eyes, human, woman, men, person, skin, shadow']

In [4]:
%%capture
for raw_sample in os.listdir(PATH_TO_STABLE_INPUT):
    imgPath = os.path.join(PATH_TO_STABLE_INPUT, raw_sample)
    init_image = Image.open(imgPath).convert("RGB")
    init_image = init_image.resize((500, 500))

    for prompt in prompts:
        images = pipe(prompt=prompt, negative_prompt=negative_prompt[0], image=init_image, strength=1, guidance_scale=7).images
        current_timestamp = datetime.datetime.now()
        formatted_timestamp = current_timestamp.strftime("%Y%m%d%H%M%S")  # Format the timestamp as a string
    
        name = clothType + formatted_timestamp + "_0.png"
        images[0].save(os.path.join(PATH_TO_STABLE_OUTPUT, name))

        # generations
        for i in range(0, 2):
                mirr_image = images[0].transpose(Image.FLIP_LEFT_RIGHT)
                # Mirror the image horizontally (left-to-right)
                image_var = pipe(prompt=prompt, 
                              negative_prompt=negative_prompt[0], 
                              image=mirr_image, 
                              strength=0.7, 
                              guidance_scale=(random.randint(5,7))
                              ).images
                name = clothType + formatted_timestamp + "_" + str(i+1) + ".png"
                image_var[0].save(os.path.join(PATH_TO_STABLE_OUTPUT, name))

KeyboardInterrupt: 